In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#!pip install psycopg2-binary
#!pip install scikit-bio
import psycopg2

sql_username = 'postgres'
database='postgres'
sql_password = '<pswd>'
host = '<host>.us-east-1.rds.amazonaws.com'

conn_v = psycopg2.connect(host = host,
                           port=5432,
                          database=database,
                          user=sql_username,
                          password=sql_password)
conn_v

In [ ]:
# get mapping file and set the column with sample names you wish to pull into a list
meta= pd.read_csv('MATCHED_case_control_cohorts.csv')
sampleids = meta['items_set_item'].values.tolist()

In [ ]:
# EXAPLE QUERY
species_query = f"""\
SELECT s.sample_name,  MAX(a.analysis_id) ,  ttw.species_name, SUM(mgr.readcount)  as readcount  from sample s
INNER JOIN analysis a
ON a.sample_id=s.sample_id
INNER JOIN microbiome_genome_results mgr
ON mgr.analysis_id=a.analysis_id
INNER JOIN microbe_genome mg
ON mg.genome_id=mgr.genome_id
INNER JOIN taxonomy_tree_wide ttw
ON ttw.assembly_accession_name=mg.genome_name
WHERE mg.microbe_genome_catalog_id='7' and ttw.taxonomy_tree_id='7'  AND s.sample_name IN {tuple(sampleids)}
GROUP BY s.sample_name,  a.analysis_id ,  ttw.species_name ; 
"""

In [ ]:
# get species level counts --- genomes will be grouped by species
species_counts= pd.read_sql(species_query, conn_v)

In [ ]:
#types of organization
species_counts = species_counts.drop(columns='max')
species_counts_pivot = species_counts.pivot(index='sample_name', values='readcount', columns='species_name')
species_counts_pivot = species_counts_pivot.fillna(0)
species_counts_pivot.to_csv('species_count.csv')
conn_v.close()

In [ ]:
# add microstat module
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

%matplotlib inline

import io
import os
import sys
print(sys.path)
sys.path.append(r'/home/ubuntu/ibd/microstat')
print(sys.path)
import microstat
from microstat.util import *
from microstat.microstats import *
from microstat.plots import *
from microstat.microml import *